In [1]:
# Standard Library
import os
from glob import glob

# Others
import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
from matplotlib.collections import LineCollection
import matplotlib as mpl

import faamasd as asd

import warnings
warnings.simplefilter("ignore", category=FutureWarning)
# catches FutureWarning in xr only

In [6]:
base = '/home/users/erinraif/mphase_data/probe_calibration/'

In [15]:
faam_pcasp_calibration_file = '/badc/faam/data/2022/c331-nov-03/core_processed/core-cloud-phy_faam_20221103_v002_r2_c331_pcasp-2_cal.nc'
faam_cdp_calibration_file = '/badc/faam/data/2022/c331-nov-03/core_processed/core-cloud-phy_faam_20221103_v000_r0_c331_cdp-1_cal.nc'
pcasp_cal_csv = base + 'pcasp_calibration.csv'
cdp_cal_csv = base + 'cdp_calibration.csv'

In [12]:
pcasp_scattering_inputs = asd.pcasp_scattering_inputs
cdp_scattering_inputs = asd.cdp_scattering_inputs
mie_scattering_folder = base + 'scattering_files'
cdp_channel_data_folder = base + 'channel_data_CDP'
pcasp_channel_data_folder = base + 'channel_data_PCASP'
attribute_file = base + 'attributes_for_calibration_arrays.txt' # Set to None if not present

In [7]:
timings_data = pd.read_csv(
    '/home/users/erinraif/acao_data/metadata/timings_of_filter_legs_acao.csv',
    index_col='unique_ID')
timings_data = timings_data[timings_data['psd_available'] == True]
timings_data = timings_data.drop(['c276r3t','c278r3t'])

In [8]:
leg_types = pd.read_csv('/home/users/erinraif/acao_data/metadata/filter_leg_types.csv',
                        index_col='unique_ID')

## Create Mie scattering data and calculate calibrated channel diameters
This is ony necessary once and requires the install of [MieConScat](https://sourceforge.net/projects/mieconscat/) and [CStoDConverter](https://sourceforge.net/projects/cstodconverter/). Methods described in Rosenberg, et al. 2012. *Particle sizing calibration with refractive index correction for light scattering optical particle counters and impacts upon PCASP and CDP data collected during the Fennec campaign.* Atmos. Meas. Tech.

In [13]:
MieConScat_prog_loc = base.replace('mphase','acao') +'MieConScat/MieConScatConsole'
CStoDConv_prog_loc = base.replace('mphase','acao') + 'CStoDConverter/cstodconverterconsole'

In [16]:
asd.create_calibration_CSV(faam_pcasp_calibration_file,
                           'PCASP',
                           pcasp_cal_csv,
                           time_index=1,
                           group=None)
asd.create_calibration_CSV(faam_cdp_calibration_file,
                           'CDP',
                           cdp_cal_csv,
                           time_index=0,
                           group=None)

/home/users/erinraif/mphase_data/probe_calibration/pcasp_calibration.csv created
/home/users/erinraif/mphase_data/probe_calibration/cdp_calibration.csv created


In [18]:
"""Note that due to restrictions on the notebook server I was using, I
ran the commands generated by generate_scattering_table directly in the shell
as I could not run the program using os.system().
"""
refractive_indices = [
    1.56+0j, 1.5+0j, 1.6+0j, 1.7+0j,
    1.56+0.001j,1.56+0.003j,1.56+0.006j,1.56+0.01j
    ]
for ri in refractive_indices:
    pcasp_mie = asd.generate_scattering_table(ri, mie_scattering_folder,
                                              MieConScat_prog_loc,
                                              pcasp_scattering_inputs)
    cdp_mie = asd.generate_scattering_table(ri, mie_scattering_folder,
                                            MieConScat_prog_loc,
                                            cdp_scattering_inputs)

/home/users/erinraif/acao_data/probe_calibration/MieConScat/MieConScatConsole -wav 0.6328 -dmin 0.05 -dmax 8 -dint 0.001 -rerimin 1.56 -imrimin 0.0 -ang1min 35 -ang1max 120 -ang2min 60 -ang2max 145 /home/users/erinraif/mphase_data/probe_calibration/scattering_files/scattering_PCASP_1.56+0j.csv
/home/users/erinraif/acao_data/probe_calibration/MieConScat/MieConScatConsole -wav 0.658 -dmin 1 -dmax 100 -dint 0.1 -rerimin 1.56 -imrimin 0.0 -ang1min 1.7 -ang1max 14 /home/users/erinraif/mphase_data/probe_calibration/scattering_files/scattering_CDP_1.56+0j.csv
/home/users/erinraif/acao_data/probe_calibration/MieConScat/MieConScatConsole -wav 0.6328 -dmin 0.05 -dmax 8 -dint 0.001 -rerimin 1.5 -imrimin 0.0 -ang1min 35 -ang1max 120 -ang2min 60 -ang2max 145 /home/users/erinraif/mphase_data/probe_calibration/scattering_files/scattering_PCASP_1.5+0j.csv
/home/users/erinraif/acao_data/probe_calibration/MieConScat/MieConScatConsole -wav 0.658 -dmin 1 -dmax 100 -dint 0.1 -rerimin 1.5 -imrimin 0.0 -ang1

/home/users/erinraif/acao_data/probe_calibration/MieConScat/MieConScatConsole: error while loading shared libraries: libgfortran.so.3: cannot open shared object file: No such file or directory
/home/users/erinraif/acao_data/probe_calibration/MieConScat/MieConScatConsole: error while loading shared libraries: libgfortran.so.3: cannot open shared object file: No such file or directory
/home/users/erinraif/acao_data/probe_calibration/MieConScat/MieConScatConsole: error while loading shared libraries: libgfortran.so.3: cannot open shared object file: No such file or directory
/home/users/erinraif/acao_data/probe_calibration/MieConScat/MieConScatConsole: error while loading shared libraries: libgfortran.so.3: cannot open shared object file: No such file or directory
/home/users/erinraif/acao_data/probe_calibration/MieConScat/MieConScatConsole: error while loading shared libraries: libgfortran.so.3: cannot open shared object file: No such file or directory
/home/users/erinraif/acao_data/prob

In [19]:
# Select only csvs with diameter data (rather than existing ones with area/volume)
diameter_csvs = glob(os.path.join(mie_scattering_folder,'*j.csv'))
for csv in diameter_csvs:
    asd.create_higher_order_scatter_data(mie_scattering_folder,os.path.basename(csv))

In [20]:
# This takes a minute or so
mie_csvs = glob(os.path.join(mie_scattering_folder,'*PCASP*.csv'))
asd.run_CStoD(pcasp_cal_csv, mie_csvs,
          CStoDConv_prog_loc,
          pcasp_channel_data_folder)
mie_csvs = glob(os.path.join(mie_scattering_folder,'*CDP*.csv'))
asd.run_CStoD(cdp_cal_csv, mie_csvs,
          CStoDConv_prog_loc,
          cdp_channel_data_folder)

/home/users/erinraif/acao_data/probe_calibration/CStoDConverter/cstodconverterconsole /home/users/erinraif/mphase_data/probe_calibration/pcasp_calibration.csv /home/users/erinraif/mphase_data/probe_calibration/scattering_files/scattering_PCASP_1.56+0j.csv /home/users/erinraif/mphase_data/probe_calibration/channel_data_PCASP/channel_data_PCASP_1.56+0j.csv
Channel 1 completed
Channel 2 completed
Channel 3 completed
Channel 4 completed
Channel 5 completed
Channel 6 completed
Channel 7 completed
Channel 8 completed
Channel 9 completed
Channel 10 completed
Channel 11 completed
Channel 12 completed
Channel 13 completed
Channel 14 completed
Channel 15 completed
Channel 16 completed
Channel 17 completed
Channel 18 completed
Channel 19 completed
Channel 20 completed
Channel 21 completed
Channel 22 completed
Channel 23 completed
Channel 24 completed
Channel 25 completed
Channel 26 completed
Channel 27 completed
Channel 28 completed
Channel 29 completed
Channel 30 completed
/home/users/erinraif/a

## Creation of particle-size distributions
Choose to use 1.56+0j as per Sanchez-Marroquin, et al. 2019 AMT

In [22]:
pcasp_calibration = asd.produce_calibration_dataset(pcasp_channel_data_folder,pcasp_scattering_inputs,
                                                    attribute_file)
cdp_calibration = asd.produce_calibration_dataset(cdp_channel_data_folder,cdp_scattering_inputs,attribute_file)

IndexError: list index out of range

In [9]:
pcasp_cal_at_ri = pcasp_calibration.sel(refractive_index = 1.56+0j)
cdp_cal_at_ri = cdp_calibration.sel(refractive_index = 1.56+0j)


In [10]:
def get_timings(metadata_row):
    """Retrieve filter run timings, including pauses.
    
    Returns a list of consecutive times as ints in SPM. This is in the format
    [start, (pause1 start), (pause1 end), ..., end]
    where pauses are only present where necessary.
    
    Parameters
    ----------
    metadata_row: pandas Series
        row of the metadata pandas dataframe
    
    Returns
    -------
    list of ints
        times in SPM of run events
    """
    leg_start_times = [metadata_row.start_time]
    leg_end_times = [metadata_row.end_time]
    pauses = metadata_row.no_pauses
    for i in range(pauses):
        start_str = 'metadata_row.pause' + str(i+1) + '_end'
        end_str = 'metadata_row.pause' + str(i+1) + '_start'
        leg_start_times.append(eval(start_str))
        leg_end_times.insert(i, eval(end_str))
    return leg_start_times, leg_end_times

In [13]:
psd_sums = pd.DataFrame(columns=['run_ID', 'dN','dS','dV','dN_err','dS_err','dV_err'])

In [14]:
for run in timings_data.itertuples():
    # Get the correct datasets for the run and and merge them 
    core_cloud_data_fn = glob(os.path.join(
        '/badc/faam/data/2022',run.flight + '*/core_processed',run.core_cloud
    ))[0]
    core_data_fn = glob(os.path.join(
        '/badc/faam/data/2022',run.flight + '*/core_processed',run.core_faam
    ))[0]
    nev_data_fn = '/home/users/erinraif/acao_data/nevzorov_data/' + run.nev_data
    flight_data = asd.get_data(core_cloud_data_fn, core_data_fn)

    # Make Nevzorov cloud-flag data compatible with other datasets
    nev_ds = xr.open_dataset(nev_data_fn, engine='netcdf4',decode_times=False)
    nev_ds = nev_ds.rename_dims({'TIME': 'time'})
    nev_ds = nev_ds.rename({'TIME': 'time'})
    nev_flag = nev_ds.CLRFLG_COMBINED

    # Get timings for each run and clip data appropriately
    leg_start_times, leg_end_times = get_timings(run)
    nev_flag = asd.time_slice_data(leg_start_times, leg_end_times, nev_flag)
    uncorrected_pcasp, corrected_pcasp, pcasp_flow = asd.get_pcasp_data_for_leg(flight_data, leg_start_times, leg_end_times)
    pcasp_psds = asd.get_mean_log_psds(pcasp_cal_at_ri, uncorrected_pcasp, corrected_pcasp, pcasp_flow)
    uncorrected_cdp, corrected_cdp, cdp_flow = asd.get_cdp_data_for_leg(flight_data, leg_start_times, leg_end_times)
    rh_liq = asd.time_slice_data(leg_start_times, leg_end_times, flight_data['RH_LIQ'])
    
    # Apply humidity mask and Nevzorov cloud-presence mask
    rh_mask = rh_liq < 80
    nev_mask = nev_flag > 0.5
    comb_mask = rh_mask & nev_mask
    comb_uncorrected_cdp = uncorrected_cdp.where(comb_mask,drop=True)
    comb_corrected_cdp = corrected_cdp.where(comb_mask, drop=True)

    # Get mean particle size distributions during each leg
    comb_cdp_psds = asd.get_mean_log_psds(cdp_cal_at_ri, comb_uncorrected_cdp, comb_corrected_cdp, cdp_flow)
    # Integrate the particle-size distributions and 
    dN, dS, dV, dN_err, dS_err, dV_err = asd.integrate_distribution_with_errors(pcasp_psds, comb_cdp_psds)
    df_row = dict(
        run_ID = str(run.Index),
        dN = dN,
        dS = dS,
        dV = dV,
        dN_err = dN_err,
        dS_err = dS_err,
        dV_err = dV_err
    )
    one_row_df = pd.DataFrame([df_row])
    psd_sums = pd.concat([psd_sums, one_row_df], axis=0, ignore_index=True)

In [16]:
psd_sums = psd_sums.set_index('run_ID')

In [18]:
filters_data = pd.read_csv(
    '/home/users/erinraif/acao_data/inp_data/subtracted_backgrounds_v4_2ul.csv',
    index_col='temp_bin')

In [33]:
nX_df = pd.DataFrame(index=filters_data.index)

""
temp_bin
-6
-7
-8
-9
-10
-11
-12
-13
-14


In [35]:
filter_concs = filters_data[filters_data.columns[pd.Series(
        filters_data.columns).str.endswith('NT')]]
filter_errors = filters_data[filters_data.columns[pd.Series(
        filters_data.columns).str.endswith('err')]]

for run in timings_data.itertuples():
    run_ID = str(run.Index)
    filter_conc = filters_data[run_ID + '_NT'] * 1000 #convert to m-3
    filter_err = filters_data[run_ID + '_err'] * 1000 #convert to m-3

    dN = psd_sums.loc[run_ID]['dN'] * 1e6 # convert to m-3
    dN_err = psd_sums.loc[run_ID]['dN_err'] * 1e6 # convert to m-3
    dS = psd_sums.loc[run_ID]['dS'] * 1e-6 # convert to m-1
    dS_err = psd_sums.loc[run_ID]['dS_err'] * 1e-6 # convert to m-1
    dV = psd_sums.loc[run_ID]['dV'] * 1e-12 # convert to m^0
    dV_err = psd_sums.loc[run_ID]['dV_err'] * 1e-12 # convert to m^0
    
    nN = filter_conc/dN
    nN_err = np.sqrt((filter_err/filter_conc)**2 + (dN_err/dN)**2)*nN
    nS = filter_conc/dS
    nS_err = np.sqrt((filter_err/filter_conc)**2 + (dS_err/dS)**2)*nS
    nV = filter_conc/dV
    nV_err = np.sqrt((filter_err/filter_conc)**2 + (dV_err/dV)**2)*nV

    new_cols = {}
    new_cols[run_ID+'_nN'] = nN
    new_cols[run_ID+'_nN_err'] = nN_err
    new_cols[run_ID+'_nS'] = nS
    new_cols[run_ID+'_nS_err'] = nS_err
    new_cols[run_ID+'_nV'] = nV
    new_cols[run_ID+'_nV_err'] = nV_err
    nX_df = pd.concat([nX_df, pd.DataFrame(new_cols)], axis=1)

In [37]:
nX_df.to_csv('/home/users/erinraif/acao_data/inp_data/nX_calibrated_v2.csv')